# Módulo 1.1: Fundamentos de MLflow con scikit-learn

## Objetivos
- Entender los conceptos básicos de MLflow
- Configurar tracking de experimentos
- Logging de parámetros, métricas y artefactos
- Primer modelo con scikit-learn

In [ ]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print(f"MLflow version: {mlflow.__version__}")

## 1. Configuración de MLflow

MLflow tiene 4 componentes principales:
- **Tracking**: Logging de parámetros, métricas y artefactos
- **Projects**: Formato para empaquetar código reproducible
- **Models**: Formato estándar para empaquetar modelos
- **Registry**: Gestión centralizada de modelos

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

experiment_name = "sklearn-basics"
mlflow.set_experiment(experiment_name)

print(f"Experiment: {experiment_name}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

## 2. Primer Experimento: Clasificación con Iris Dataset

### Paso 1: Cargar y explorar datos

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
df['species'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

print(df.head())
print(f"\nDataset shape: {df.shape}")
print(f"Class distribution:\n{df['species'].value_counts()}")

### Paso 2: Preparar datos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

### Paso 3: Entrenar modelo CON MLflow tracking

Aquí es donde MLflow entra en acción. Usamos `mlflow.start_run()` para crear un contexto de tracking.

In [ ]:
with mlflow.start_run(run_name="logistic_regression_v1") as run:
    
    params = {
        "solver": "lbfgs",
        "max_iter": 200,
        "multi_class": "multinomial",
        "random_state": 42
    }
    
    mlflow.log_params(params)
    
    mlflow.log_param("scaler", "StandardScaler")
    mlflow.log_param("test_size", 0.2)
    
    model = LogisticRegression(**params)
    model.fit(X_train_scaled, y_train)
    
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test, average='weighted')
    recall = recall_score(y_test, y_pred_test, average='weighted')
    f1 = f1_score(y_test, y_pred_test, average='weighted')
    
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    cm = confusion_matrix(y_test, y_pred_test)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png')
    mlflow.log_artifact('confusion_matrix.png')
    plt.close()
    
    mlflow.sklearn.log_model(model, "logistic_regression_model")
    
    mlflow.set_tag("model_type", "classification")
    mlflow.set_tag("dataset", "iris")
    mlflow.set_tag("framework", "sklearn")
    
    print(f"Run ID: {run.info.run_id}")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")

## 3. Comparación de Modelos

Vamos a entrenar múltiples modelos y compararlos usando MLflow

In [ ]:
models_config = [
    {
        "name": "RandomForest_n50",
        "model": RandomForestClassifier(n_estimators=50, random_state=42),
        "params": {"n_estimators": 50, "max_depth": None}
    },
    {
        "name": "RandomForest_n100",
        "model": RandomForestClassifier(n_estimators=100, random_state=42),
        "params": {"n_estimators": 100, "max_depth": None}
    },
    {
        "name": "RandomForest_depth5",
        "model": RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42),
        "params": {"n_estimators": 50, "max_depth": 5}
    },
]

results = []

for config in models_config:
    with mlflow.start_run(run_name=config["name"]):
        
        mlflow.log_params(config["params"])
        mlflow.log_param("scaler", "StandardScaler")
        
        model = config["model"]
        model.fit(X_train_scaled, y_train)
        
        y_pred = model.predict(X_test_scaled)
        
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        mlflow.log_metric("test_accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        
        mlflow.sklearn.log_model(model, "random_forest_model")
        
        mlflow.set_tag("model_type", "RandomForest")
        
        results.append({
            "name": config["name"],
            "accuracy": accuracy,
            "f1_score": f1
        })
        
        print(f"{config['name']}: Accuracy={accuracy:.4f}, F1={f1:.4f}")

results_df = pd.DataFrame(results)
print("\nComparación de Modelos:")
print(results_df)

## 4. Logging de Artefactos Adicionales

Podemos guardar cualquier archivo: gráficos, datasets, reportes, etc.

In [ ]:
with mlflow.start_run(run_name="logistic_with_artifacts"):
    
    model = LogisticRegression(max_iter=200, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("test_accuracy", accuracy)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    for idx, feature_idx in enumerate([0, 1, 2, 3]):
        ax = axes[idx // 2, idx % 2]
        for target_class in [0, 1, 2]:
            ax.hist(X[y == target_class, feature_idx], alpha=0.5, 
                   label=iris.target_names[target_class], bins=20)
        ax.set_xlabel(iris.feature_names[feature_idx])
        ax.set_ylabel('Frequency')
        ax.legend()
    
    plt.tight_layout()
    plt.savefig('feature_distributions.png')
    mlflow.log_artifact('feature_distributions.png')
    plt.close()
    
    feature_importance_df = pd.DataFrame({
        'feature': iris.feature_names,
        'coefficient_class_0': model.coef_[0],
        'coefficient_class_1': model.coef_[1],
        'coefficient_class_2': model.coef_[2]
    })
    feature_importance_df.to_csv('feature_importance.csv', index=False)
    mlflow.log_artifact('feature_importance.csv')
    
    mlflow.sklearn.log_model(model, "model_with_artifacts")
    
    print(f"Accuracy: {accuracy:.4f}")
    print("\nFeature Coefficients:")
    print(feature_importance_df)

## 5. Cargar un Modelo desde MLflow

Una vez guardado, podemos cargar el modelo en cualquier momento

In [ ]:
experiment = mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

print("Runs disponibles:")
print(runs[['run_id', 'tags.mlflow.runName', 'metrics.test_accuracy']].head())

best_run = runs.loc[runs['metrics.test_accuracy'].idxmax()]
best_run_id = best_run['run_id']

print(f"\nMejor run: {best_run['tags.mlflow.runName']}")
print(f"Run ID: {best_run_id}")
print(f"Accuracy: {best_run['metrics.test_accuracy']:.4f}")

In [ ]:
logged_model = f'runs:/{best_run_id}/logistic_regression_model'

loaded_model = mlflow.sklearn.load_model(logged_model)

sample = X_test_scaled[:5]
predictions = loaded_model.predict(sample)

print("Predicciones con modelo cargado:")
for i, pred in enumerate(predictions):
    print(f"Sample {i+1}: Predicted={iris.target_names[pred]}, Actual={iris.target_names[y_test[i]]}")

## Resumen del Módulo 1.1

### Conceptos Clave Aprendidos:

1. **Configuración de MLflow**
   - `mlflow.set_tracking_uri()`: Configurar servidor de tracking
   - `mlflow.set_experiment()`: Crear/seleccionar experimento

2. **Logging de Información**
   - `mlflow.log_param()`: Parámetros del modelo
   - `mlflow.log_metric()`: Métricas de performance
   - `mlflow.log_artifact()`: Archivos (gráficos, CSVs, etc.)
   - `mlflow.set_tag()`: Metadatos adicionales

3. **Gestión de Modelos**
   - `mlflow.sklearn.log_model()`: Guardar modelo
   - `mlflow.sklearn.load_model()`: Cargar modelo

4. **Búsqueda de Experimentos**
   - `mlflow.search_runs()`: Buscar y comparar runs

### Siguiente Paso:
En el siguiente notebook trabajaremos con pipelines de clasificación más complejos.